In [ ]:
!gdown 1rTjraSCHo63587i1ZvB_AdmKjyE1m-LW
#!gdown 16oGUmzV9b7GYv6pgl__8xG9uyXwY3_t2
!unzip t2_redes_neurais.zip
!rm -rf sample_data
!mv t2_redes_neurais/* .

# DO DRIVE PRO COLAB
#import shutil
#import os
#
#source_path = '/content/drive/MyDrive/t2_redes_neurais'
#dest_path = '/content/'
#
## Garante que a pasta de destino exista
#os.makedirs(dest_path, exist_ok=True)
#
## Copia todos os arquivos e subpastas individualmente
#for item in os.listdir(source_path):
#    s = os.path.join(source_path, item)
#    d = os.path.join(dest_path, item)
#    if os.path.isdir(s):
#        shutil.copytree(s, d, dirs_exist_ok=True)
#    else:
#        shutil.copy2(s, d)
#
#!rm -rf sample_data/

Downloading...
From (original): https://drive.google.com/uc?id=1rTjraSCHo63587i1ZvB_AdmKjyE1m-LW
From (redirected): https://drive.google.com/uc?id=1rTjraSCHo63587i1ZvB_AdmKjyE1m-LW&confirm=t&uuid=5ef14bce-eb74-4676-bb39-a7dca75570b3
To: /content/t2_redes_neurais.zip
100% 1.08G/1.08G [00:15<00:00, 70.7MB/s]
Archive:  t2_redes_neurais.zip
   creating: t2_redes_neurais/
   creating: t2_redes_neurais/.ipynb_checkpoints/
   creating: t2_redes_neurais/annotations/
  inflating: t2_redes_neurais/annotations/annotations_unidist_train.pkl  
  inflating: t2_redes_neurais/annotations/annotations_consecutive_train.pkl  
  inflating: t2_redes_neurais/annotations/annotations_consecutive_test.pkl  
  inflating: t2_redes_neurais/annotations/annotations_unidist_test.pkl  
   creating: t2_redes_neurais/sample_data/
  inflating: t2_redes_neurais/sample_data/california_housing_train.csv  
  inflating: t2_redes_neurais/sample_data/anscombe.json  
  inflating: t2_redes_neurais/sample_data/california_housing_

In [ ]:
# DO COLAB PRO DRIVE
# %%bash

# # Cria a pasta de destino, o '-p' evita erro se ela já existir
# mkdir -p /content/drive/MyDrive/t2_redes_neurais/

# # Loop para copiar cada item em /content, exceto a pasta /content/drive
# for item in /content/*; do
#   # Compara a string do item com a string que queremos excluir
#   if [ "$item" != "/content/drive" ]; then
#     # Imprime uma mensagem para sabermos o que está sendo copiado
#     echo "Copiando $item ..."
#     # Copia o item (se for uma pasta, o -r copia o conteúdo)
#     cp -r "$item" "/content/drive/MyDrive/t2_redes_neurais/"
#   else
#     echo "Ignorando a pasta /content/drive"
#   fi
# done

# echo "Backup concluído!"

In [46]:
import os
import torch
from torch import nn
from preprocessed_dataset import PreprocessedDataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from dataloader_multimodal import VideoDataset
from opts import *
from collections import Counter

In [51]:
PROCESSED_TRAIN_DIR = './processed_data/train/'
PROCESSED_TEST_DIR = './processed_data/test/'

In [17]:
class AuralSkillClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        # inicia um backbone da rede com a ResNet18 pré-treinada
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)

        # modifica a primeira camada para receber apenas 1 canal de entrada
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # remove a camada de classificação original da ResNet, substituindo-a por uma nn.Identity()
        self.backbone.fc = nn.Identity()

        # cria uma nova camada de classificação, conforme descrita no artigo, com 512 valores de entrada e 128 de saída,
        # posteriormente passando por uma camada que faz a classificação de fato, com 10 valores de saída (correspondentes às classes)
        self.classifier = nn.Sequential(
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        self.classification_head = nn.Linear(128, num_classes)
        self.regression_head = nn.Linear(128, 1)

    def forward(self, x):
        features = self.backbone(x) # passa a entrada pela rede ResNet
        features_128 = self.classifier(features) # passa o resultado da ResNet pela camada final de classificação
        logits_cls = self.classification_head(features_128)
        output_reg = self.regression_head(features_128)

        return logits_cls, output_reg

In [43]:
def calculate_class_weights(dataset, num_classes=10):
    """
    Calcula os pesos para cada classe com base na frequência inversa das amostras.
    Esta função percorre o dataset uma vez para contar as ocorrências de cada classe
    e depois calcula os pesos.

    :param dataset: Uma instância do seu objeto de Dataset (ex: PreprocessedDataset).
    :param num_classes: O número total de classes no seu problema.
    :return: Um tensor do PyTorch de formato [num_classes] com o peso para cada classe.
    """
    print("Iniciando o cálculo dos pesos das classes...")

    # --- PASSO 1: Contar as Amostras de Cada Classe ---
    
    # A forma mais eficiente de contar é usar um DataLoader para iterar sobre os dados.
    # batch_size pode ser maior para acelerar a contagem. shuffle=False não é necessário aqui.
    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
    
    # Usaremos um Counter para armazenar as contagens de cada label.
    class_counts = Counter()
    
    # Itera sobre o dataset para contar os labels
    for batch_data in loader:
        labels = batch_data['player_lvl']
        # .tolist() converte o tensor de labels do lote para uma lista Python
        class_counts.update(labels.tolist())

    # Transforma o Counter em uma lista ordenada pelo índice da classe (de 0 a 9)
    # Se uma classe não aparecer, sua contagem será 0.
    counts = [class_counts.get(i, 0) for i in range(num_classes)]
    print(f"Contagem de amostras por classe: {counts}")
    

    # --- PASSO 2: Calcular os Pesos ---
    
    # A fórmula é o inverso da frequência: peso = 1 / contagem
    # Usamos uma lista para guardar os pesos calculados.
    weights = []
    for count in counts:
        # Lida com o caso de uma classe não ter amostras para evitar divisão por zero
        if count == 0:
            weights.append(0.0)
        else:
            weights.append(1.0 / count)

    # Converte a lista de pesos em um tensor do PyTorch do tipo float
    weights_tensor = torch.tensor(weights, dtype=torch.float)
    
    # Opcional, mas recomendado: Normalizar os pesos para que a soma deles não seja muito grande,
    # o que poderia desestabilizar o treinamento. Aqui, normalizamos pela soma.
    weights_tensor = weights_tensor / weights_tensor.sum()
    
    print(f"Pesos calculados para as classes: {weights_tensor}")
    print("Cálculo de pesos concluído.")
    
    return weights_tensor


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo: {device}")

# 2. Crie uma instância do seu modelo e mova para o dispositivo
model = AuralSkillClassifier(num_classes=10).to(device)

# Coloca o modelo em modo de avaliação (importante para desativar o Dropout)
model.eval()

# 3. Crie um lote de dados "falsos" (dummy tensor)
# Formato: [batch_size, channels, height, width]
# batch_size = 4 (simulando 4 imagens de uma vez)
# channels = 1 (seu espectrograma é escala de cinza)
# height = 224
# width = 224
batch_size = 4
dummy_input = torch.rand(batch_size, 1, 224, 224).to(device)

print(f"Formato do tensor de entrada: {dummy_input.shape}")

# 4. Passe os dados falsos pelo modelo
# torch.no_grad() desliga o cálculo de gradientes, o que torna
# a inferência mais rápida e economiza memória.
with torch.no_grad():
    output = model(dummy_input)

# 5. Verifique a saída
print(f"Formato do tensor de saída: {output[0].shape}, {output[1].shape}")
print("Tensor de saída (logits):")
print(output)

Usando o dispositivo: cuda
Formato do tensor de entrada: torch.Size([4, 1, 224, 224])
Formato do tensor de saída: torch.Size([4, 10]), torch.Size([4, 1])
Tensor de saída (logits):
(tensor([[-0.0118, -0.0955,  0.2638,  0.0963, -0.2524,  0.0361,  0.0321,  0.1102,
         -0.0174, -0.0822],
        [-0.0173, -0.0901,  0.2540,  0.1013, -0.2361,  0.0285,  0.0179,  0.1086,
         -0.0186, -0.0912],
        [ 0.0053, -0.0950,  0.2663,  0.0904, -0.2217,  0.0257,  0.0333,  0.1043,
         -0.0123, -0.0802],
        [-0.0085, -0.0888,  0.2559,  0.1078, -0.2370,  0.0417,  0.0287,  0.1024,
         -0.0082, -0.0786]], device='cuda:0'), tensor([[0.2636],
        [0.2657],
        [0.2642],
        [0.2775]], device='cuda:0'))


In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# train_dataset = VideoDataset(mode='train')
train_dataset = PreprocessedDataset(data_dir=PROCESSED_TRAIN_DIR)

Dataset encontrado. Número de amostras: 516
Dataset encontrado. Número de amostras: 476


In [47]:
pesos_tensor = calculate_class_weights(train_dataset).to(device)

Iniciando o cálculo dos pesos das classes...
Contagem de amostras por classe: [18, 19, 34, 16, 27, 22, 72, 89, 157, 62]
Pesos calculados para as classes: tensor([0.1682, 0.1594, 0.0891, 0.1893, 0.1122, 0.1377, 0.0421, 0.0340, 0.0193,
        0.0488])
Cálculo de pesos concluído.


In [48]:
criterion_cls = nn.CrossEntropyLoss(weight=pesos_tensor)
criterion_reg_l1 = nn.L1Loss()
criterion_reg_l2 = nn.MSELoss()

In [ ]:
# Crie o DataLoader (você já deve ter feito isso em um passo anterior)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

In [ ]:
import os
import torch

# Caminho para o modelo pré-treinado
checkpoint_dir = './checkpoints_batch4_weight_decay/'
checkpoint_path = './chechpoints_batch4_weight_decay'
# checkpoint_path = os.path.join('./checkpoints', 'model_epoch_100.pt')

# Configurações do modelo e do otimizador (defina o seu modelo, critério e otimizador aqui)
# DESCOMENTAR
# Exemplo de inicialização do modelo, critério e otimizador (substitua pelos seus):
# model = YourModel()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion_cls = torch.nn.CrossEntropyLoss()  # Para classificação
# criterion_reg_l1 = torch.nn.L1Loss()  # Para regressão
# criterion_reg_l2 = torch.nn.MSELoss()  # Para regressão

# Tentar carregar o modelo e o otimizador a partir do checkpoint
# if os.path.exists(checkpoint_path):
#     print(f"Checkpoint encontrado. Carregando o modelo pré-treinado de {checkpoint_path}...")
#     checkpoint = torch.load(checkpoint_path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     start_epoch = checkpoint['epoch']
#     loss = checkpoint['loss']  # Perda final do último lote da época
#     print(f"Modelo carregado com sucesso a partir da época {start_epoch}. Continuando o treinamento...")
# else:
print("Nenhum checkpoint encontrado. Iniciando treinamento do zero.")
start_epoch = 0  # Começar do início

# --- INÍCIO DO TREINAMENTO ---

# Coloque o modelo no modo de treinamento
model.train()

# Número de épocas de treinamento
num_epochs = 100

for epoch in range(start_epoch, num_epochs):
    print(f"--- Iniciando Época {epoch+1}/{num_epochs} ---")

    # Para cada lote de dados
    for i, batch_data in enumerate(train_loader):
        print(f"  Processando lote {i+1}/{len(train_loader)}")

        optimizer.zero_grad()

        spectrograms_tensor = batch_data['audio'].to(device)
        labels = batch_data['player_lvl'].to(device)

        lista_outputs_cls = []
        lista_outputs_reg = []

        labels_long = labels.long()
        labels_float = labels.float()

        # Processando cada clipe
        for i in range(nclips):
            clip_tensor = spectrograms_tensor[:, i, :, :, :]
            logits_cls_clip, output_reg_clip = model(clip_tensor)
            lista_outputs_cls.append(logits_cls_clip)
            lista_outputs_reg.append(output_reg_clip)

        # Calculando a saída média
        logits_cls = torch.stack(lista_outputs_cls).mean(dim=0)
        output_reg = torch.stack(lista_outputs_reg).mean(dim=0)

        # Calculando a perda
        loss_cls = criterion_cls(logits_cls, labels_long)
        output_reg = output_reg.squeeze()
        l1 = criterion_reg_l1(output_reg, labels_float)
        l2 = criterion_reg_l2(output_reg, labels_float)
        loss_reg = l1 + l2

        # Perda total
        loss = (1.0 * loss_cls) + (0.1 * loss_reg)

        # Backpropagation
        loss.backward()
        optimizer.step()

    # --- Bloco de Salvamento ao Final de Cada Época ---
    print(f"--- Fim da Época {epoch+1}. Salvando checkpoint... ---")

    # Crie uma pasta para os checkpoints no seu Drive
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Defina o caminho completo para o arquivo do checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch+1}.pt')

    # Crie o dicionário com tudo que você quer salvar
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,  # Salva a perda do último lote da época
    }, checkpoint_path)

    print(f"Checkpoint salvo em: {checkpoint_path}")


Nenhum checkpoint encontrado. Iniciando treinamento do zero.
--- Iniciando Época 1/100 ---
  Processando lote 1/129
  Processando lote 2/129
  Processando lote 3/129
  Processando lote 4/129
  Processando lote 5/129
  Processando lote 6/129
  Processando lote 7/129
  Processando lote 8/129
  Processando lote 9/129
  Processando lote 10/129
  Processando lote 11/129
  Processando lote 12/129
  Processando lote 13/129
  Processando lote 14/129
  Processando lote 15/129
  Processando lote 16/129
  Processando lote 17/129
  Processando lote 18/129
  Processando lote 19/129
  Processando lote 20/129
  Processando lote 21/129
  Processando lote 22/129
  Processando lote 23/129
  Processando lote 24/129
  Processando lote 25/129
  Processando lote 26/129
  Processando lote 27/129
  Processando lote 28/129
  Processando lote 29/129
  Processando lote 30/129
  Processando lote 31/129
  Processando lote 32/129
  Processando lote 33/129
  Processando lote 34/129
  Processando lote 35/129
  Proces

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error
import numpy as np

# Garante que o modelo está em modo de avaliação (desliga dropout, etc.)
model.eval()

# Cria o dataset de teste para pegar uma amostra
# (Certifique-se que o modo 'test' corresponde ao seu arquivo .pkl de teste)
try:
    test_dataset = PreprocessedDataset(PROCESSED_TEST_DIR)
    test_loader = DataLoader(test_dataset, batch_size=64, num_workers=4)
    print(f"Dataset de teste carregado. Total de amostras: {len(test_dataset)}")
except Exception as e:
    print(f"Erro ao carregar o dataset de teste: {e}")
    print("Verifique se o arquivo 'annotations_unidist_test.pkl' existe na sua pasta de anotações.")
    # Se der erro aqui, pare a execução da célula
    raise

# Armazenar as previsões e os rótulos verdadeiros
all_predictions = []
all_true_labels = []
cnt = 0
# Loop para avaliar todo o dataset de teste
for batch_data in test_loader:
    cnt += 1
    print(cnt)
    # Prepara os dados para o modelo
    input_tensor = batch_data['audio'].to(device)
    true_labels_batch = batch_data['player_lvl']
    # input_tensor = batch_data['audio'].unsqueeze(0).to(device)
    # true_label = batch_data['player_lvl']

    # Bloco de inferência sem cálculo de gradientes para economizar memória e ser mais rápido
    with torch.no_grad():

        # --- Lógica de inferência para múltiplos clipes (mesma do treino) ---
        clip_outputs_cls = []
        n_clips_from_tensor = input_tensor.shape[1] # Pega o nclips do próprio tensor

        for i in range(n_clips_from_tensor):
            # Pega o i-ésimo clipe
            clip_tensor = input_tensor[:, i, :, :, :]

            # Passa o clipe pelo modelo
            logits_cls_clip, _ = model(clip_tensor)

            # Guarda a saída de classificação
            clip_outputs_cls.append(logits_cls_clip)

        # Agrega as saídas dos clipes tirando a média
        final_logits = torch.stack(clip_outputs_cls).mean(dim=0)
        # --- Fim da lógica de inferência ---

        # Converte os logits em probabilidades
        probabilities = torch.softmax(final_logits, dim=1)

        # Pega a previsão com a maior probabilidade
        # predicted_index = torch.argmax(probabilities, dim=1).item()
        predicted_index = torch.argmax(probabilities, dim=1)


    # Armazena as previsões e os rótulos verdadeiros
    all_predictions.extend(predicted_index.cpu().numpy())
    all_true_labels.extend(true_labels_batch.cpu().numpy())

# --- Cálculo das Métricas ---
accuracy = accuracy_score(all_true_labels, all_predictions)
precision = precision_score(all_true_labels, all_predictions, average='weighted', zero_division=1)
recall = recall_score(all_true_labels, all_predictions, average='weighted', zero_division=1)
f1 = f1_score(all_true_labels, all_predictions, average='weighted', zero_division=1)
mae = mean_absolute_error(all_true_labels, all_predictions)

# Exibe as métricas
print("="*40)
print(f"--- RESULTADOS DA AVALIAÇÃO NO CONJUNTO DE TESTE ---")
print("="*40)
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision (weighted): {precision*100:.2f}%")
print(f"Recall (weighted): {recall*100:.2f}%")
print(f"F1 Score (weighted): {f1*100:.2f}%")
print(f"Mean Average Error (MAE): {mae:.2f}")
print("="*40)

Dataset encontrado. Número de amostras: 476
Dataset de teste carregado. Total de amostras: 476
1
2
3
4
5
6
7
8
--- RESULTADOS DA AVALIAÇÃO NO CONJUNTO DE TESTE ---
Accuracy: 60.50%
Precision (weighted): 79.26%
Recall (weighted): 60.50%
F1 Score (weighted): 62.22%
Mean Average Error (MAE): 1.14
